Objective:
1. setup langchain, langsmith
2. create simple application with langchain
    - use prompt templates, models, output parsers
    - use lcel
3. start tracing with langsmith


Basics:
- Primitives like llms, parsers, prompts, retrievers, agents -> implement LangChain Runnable Interface

### simple chain

In [1]:
! pip install langchain --quiet

Object "install" is unknown, try "ip help".


In [80]:
import json

with open("/home/ubuntu/config.json") as file:
    config = json.load(file)

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=config["langsmith_api_key"]

In [12]:
from langchain_core.prompts import (
    PromptTemplate, ChatPromptTemplate 
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer"),
    ("user", "{input}")
])

# prompt.messages[0].pretty_print()

In [13]:
import boto3
from langchain_community.chat_models import BedrockChat

llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    # model_id="anthropic.claude-3-haiku-20240307-v1:0",
    # model_id="anthropic.claude-instant-v1",
    # model_id="anthropic.claude-v2:1",

    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        "max_tokens": 128,
        # "max_tokens_to_sample": 128,
        "temperature": 0.5,
    }
)

In [4]:
chain = prompt | llm

In [5]:
chain.invoke({"input": "How can Langsmith help with testing ?"})

AIMessage(content="Langsmith can be a valuable tool for testing software applications and systems. Here are some ways Langsmith can assist with testing:\n\n1. **Test Case Generation**: Langsmith can be used to generate test cases automatically based on the requirements or specifications of the software. It can analyze the input data, expected outputs, and edge cases to create comprehensive test scenarios.\n\n2. **Natural Language Test Scripting**: With Langsmith's natural language processing capabilities, testers can write test scripts using plain English or other natural languages. This can make test script creation more accessible and easier to understand, especially for non-technical")

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()

In [7]:
print(chain.invoke({"input": "How can Langsmith help with testing ?"}))

Langsmith can be a valuable tool for testing, particularly when it comes to testing software systems that involve natural language processing (NLP) or generation. Here are some ways Langsmith can assist with testing:

1. **Test Data Generation**: Langsmith can generate large volumes of realistic and varied test data, including natural language text, dialogues, and scenarios. This can be useful for testing the robustness and edge cases of NLP systems, chatbots, or language models.

2. **Test Case Generation**: Langsmith can help generate test cases for NLP systems by creating a diverse set of


### add context -> retrieval_chain

Thought: add context using a retriever (which contains text scraped from web)

Steps:
1. create documents (scrape from web)
2. index documents
3. create a retriever
4. write q&a prompt
5. create retrieval chain


In [8]:
! pip install beautifulsoup4 --quiet

In [15]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

# print(docs[0].page_content)

In [20]:
! pip install cohere --quiet
! pip install faiss-cpu --quiet

In [16]:
import os
os.environ["COHERE_API_KEY"] = config["cohere_api_key"]

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import CohereEmbeddings

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

embeddings = CohereEmbeddings()

In [18]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

In [25]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [30]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [31]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'context', 'input'] Received: ['input', 'context']"

In [29]:
response

{'input': 'how can langsmith help with testing?',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for t

In [28]:
print(response["answer"])

According to the context provided, LangSmith can help with testing LLM applications in the following ways:

1. Initial Test Set: LangSmith allows developers to create datasets which are collections of inputs and reference outputs. These datasets can be used to run tests on their LLM applications. The test cases can be uploaded in bulk, created on the fly, or exported from application traces.

2. Comparison View: When making changes to the application, LangSmith provides a user-friendly comparison view that allows side-by-side comparison of test results across multiple revisions of the application. This


### conversational retrieval chain

In [24]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        (
            "user", 
            "{input}"
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation"
        )
    ]
)

# prompt.messages[2].pretty_print()

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [26]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(
        # name="H", 
        content="Can LangSmith help test my LLM applications?"
    ), 
    AIMessage(
        # name="A", 
        content="Yes!"
    )    
]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications,

In [32]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [33]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understan

### agents

In [35]:
! pip install langchainhub --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.34.30 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.2.1 which is incompatible.
sagemaker 2.205.0 requires urllib3<1.27, but you have urllib3 2.2.1 which is incompatible.


In [52]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [58]:
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = config["tavily_api_key"]
search = TavilySearchResults(max_results=1)

# name: str = "tavily_search_results_json"
# description: str = (
#     "A search engine optimized for comprehensive, accurate, and trusted results. "
#     "Useful for when you need to answer questions about current events. "
#     "Input should be a search query."
# )

In [59]:
tools = [retriever_tool, search]

tools

[Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f3f282a9480>, retriever=VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f3f2a94ac80>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7f3f282a9510>, retriever=VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f3f2a94ac80>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 TavilySearchResults(max_results=1)]

In [73]:
# from langchain import hub

# # prompt = hub.pull("hwchase17/openai-functions-agent")
# # prompt = hub.pull("hwchase17/react")
# prompt = hub.pull("hwchase17/react-chat")
# prompt.pretty_print()


In [75]:
prompt_template = """Assistant is a large language model trained by Anthropic.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}

"""


prompt = PromptTemplate(
    input_variables=[
        "tools", "tool_names", "chat_history", "input", "agent_scratchpad"
    ],
    template=prompt_template
)

In [76]:
import boto3
from langchain_community.chat_models import BedrockChat

from langchain.agents import create_react_agent
from langchain.agents import AgentExecutor


# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.pretty_print()
llm = BedrockChat(
    # model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    # model_id="anthropic.claude-instant-v1",
    # model_id="anthropic.claude-v2:1",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        "max_tokens": 128,
        # "max_tokens_to_sample": 128,
        "temperature": 0.0,
    }
)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [77]:
agent_executor.invoke({
    "input": "how can langsmith help with testing?",
    "chat_history": []
})



> Entering new AgentExecutor chain...


Thought: I should use the langsmith_search tool to find information about how LangSmith can help with testing.

Action: langsmith_search
Action Input: langsmith testing
Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model 

{'input': 'how can langsmith help with testing?',
 'chat_history': [],
 'output': 'LangSmith provides several key features to support testing of LLM applications:\n\n1. Debugging: LangSmith provides detailed tracing and visibility into each step of an LLM sequence, making it easier to identify and debug issues.\n\n2. Initial Test Sets: LangSmith allows developers to create datasets of inputs and reference outputs to run tests on their LLM applications'}

In [78]:
agent_executor.invoke({
    "input": "what is the weather in SF?",
    "chat_history": []
})



> Entering new AgentExecutor chain...


Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: weather in SF[{'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San francisco,CA', 'content': 'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire Weather; ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 8:40 pm PDT Mar 14, 2024. Forecast Valid: 12am PDT Mar 15, 2024-6pm PDT Mar 20, 2024 .'}]Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: weather in SF[{'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health 

{'input': 'what is the weather in SF?',
 'chat_history': [],
 'output': 'According to the weather information I found, the current weather in San Francisco, CA is:\n\n- Considerable cloudiness today, with light rain early on Tuesday followed by remaining cloudy conditions with showers in the afternoon. \n- Wednesday is expected to be overcast with rain showers at times.\n- The 10-day forecast shows a mix of cloudy, rainy, and cooler conditions in the San Francisco area over the next week.\n\nSo it looks like the weather in San Francisco is expected to be rather unsettled'}

In [79]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...


Thought: Do I need to use a tool? Yes
Action: langsmith_search
Action Input: LangSmithLangSmith User Guide | 🦜️🛠️ LangSmith

Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failin

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith provides several key features to help test and debug LLM applications:\n\n1. Prototyping: LangSmith allows for rapid experimentation with different prompts, models, and retrieval strategies. It provides visibility into model performance to help identify issues quickly.\n\n2. Debugging: LangSmith provides detailed tracing of LLM sequences, making it easier to identify and root-cause issues in'}